In [1]:
import numpy, gc, os
from tgnn.util import crystal_conv as cc
import tgnn.util.trainer as tr
import torch.nn
from torch.utils.data import DataLoader
from tgnn.model.TGNN import TGNN, TGNNX
from tgnn.model.CGCNN import CGCNN, CGCNNR
from tgnn.util.AdaBound import AdaBound
from torch.utils.tensorboard import SummaryWriter

root_data  = 'c:/WORKSPACE_KRICT/DATA/data_snu/with_metal'
root_model = 'd:/MODELS/202204/'


In [2]:
gc.collect()
torch.cuda.empty_cache()
num_epochs = 300
batch_size = 128

cc.load_mat_atom_feats()
list_test_mae = list()
list_test_rmse = list()

train_data, val_data, test_data = cc.load_dataset(root_data, fn='id_target.ins.csv', target_idx=3, ref_idx=1, radius=8, train_ratio=0.7, val_ratio=0.2, model_type='cgcnn')

for mtype in ['cx','cr']:
    train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=tr.collate_cgcnn)
    val_data_loader = DataLoader(val_data, batch_size=batch_size, collate_fn=tr.collate_cgcnn)
    test_data_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=tr.collate_cgcnn)

    if mtype == 'cx':
        model = CGCNN(cc.num_atom_feats, cc.num_edge_feats, 1).cuda()
        odir  = 'cgcnn_x'
    elif mtype == 'cr':
        model = CGCNNR(cc.num_atom_feats, cc.num_edge_feats, 1).cuda()
        odir  = 'cgcnn_r'

    optimizer = AdaBound(model.parameters(), lr=1e-6, weight_decay=1e-8)
    # optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-8)
    criterion = torch.nn.L1Loss()

    for i in range(99):
        output_root = os.path.join(root_model, odir, '{:02d}'.format(i))
        if not os.path.isdir(output_root): 
            os.makedirs(output_root)
            break
    writer = SummaryWriter(output_root)

    for epoch in range(0, num_epochs):
        train_loss = tr.train(model, optimizer, train_data_loader, criterion)
        val_loss, val_rmse, _, _, _ = tr.test(model, val_data_loader, criterion)
        print('Epoch [{}/{}]\tTrain loss: {:.4f}\tVal loss: {:.4f} ({:.4f})'
                .format(epoch + 1, num_epochs, train_loss, val_loss, val_rmse))

        writer.add_scalar('train/loss', train_loss, epoch)
        writer.add_scalar('train/MAE', train_loss, epoch)
        writer.add_scalar('valid/loss', val_loss, epoch)
        writer.add_scalar('valid/MAE', val_loss, epoch)

        if epoch%20 == 0:
            torch.save(model.state_dict(), os.path.join(output_root, 'model.{:05d}.pt'.format(epoch)))
        if epoch%10 == 0:
            _, _, idxs, targets, preds = tr.test(model, test_data_loader, criterion)
            numpy.savetxt(os.path.join(output_root, 'pred.{:05d}.txt'.format(epoch)), numpy.hstack([idxs, targets, preds]), delimiter=',')

100%|██████████| 10387/10387 [01:59<00:00, 87.04it/s] 
c:\WORKSPACE_KRICT\CODES\band_gap_model\notebooks\tgnn\util\AdaBound.py:91: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1050.)
  grad = grad.add(group['weight_decay'], p.data)


Epoch [1/300]	Train loss: 2.0936	Val loss: 0.9687 (1.3683)
Epoch [2/300]	Train loss: 0.8684	Val loss: 0.8347 (1.3832)
Epoch [3/300]	Train loss: 0.8185	Val loss: 0.8114 (1.3059)
Epoch [4/300]	Train loss: 0.7947	Val loss: 0.7808 (1.6831)
Epoch [5/300]	Train loss: 0.7741	Val loss: 0.7888 (1.9485)
Epoch [6/300]	Train loss: 0.7769	Val loss: 0.7653 (1.7679)
Epoch [7/300]	Train loss: 0.7562	Val loss: 0.7515 (1.7169)
Epoch [8/300]	Train loss: 0.7594	Val loss: 0.7422 (1.5756)
Epoch [9/300]	Train loss: 0.7451	Val loss: 0.7331 (1.5884)
Epoch [10/300]	Train loss: 0.7248	Val loss: 0.8297 (1.9085)
Epoch [11/300]	Train loss: 0.7326	Val loss: 0.7419 (1.4234)
Epoch [12/300]	Train loss: 0.7185	Val loss: 0.8014 (1.5301)
Epoch [13/300]	Train loss: 0.7148	Val loss: 0.7089 (1.5828)
Epoch [14/300]	Train loss: 0.7298	Val loss: 0.7122 (1.4610)
Epoch [15/300]	Train loss: 0.7009	Val loss: 0.8394 (1.4919)
Epoch [16/300]	Train loss: 0.6960	Val loss: 0.7229 (1.4868)
Epoch [17/300]	Train loss: 0.6879	Val loss: 0.727

In [3]:
gc.collect()
torch.cuda.empty_cache()
num_epochs = 300
batch_size = 32

cc.load_mat_atom_feats()
list_test_mae = list()
list_test_rmse = list()

train_data, val_data, test_data = cc.load_dataset(root_data, fn='id_target.ins.csv', target_idx=3, ref_idx=1, radius=4, train_ratio=0.7, val_ratio=0.2, model_type='tgnn')

for mtype in ['tx','tr']:
    train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=tr.collate)
    val_data_loader = DataLoader(val_data, batch_size=batch_size, collate_fn=tr.collate)
    test_data_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=tr.collate)

    if mtype == 'tx':
        model = TGNNX(cc.num_atom_feats, cc.num_bond_feats, 1).cuda()
        odir  = 'tgnn_x'
    elif mtype == 'tr':
        model = TGNN(cc.num_atom_feats, cc.num_bond_feats, 1).cuda()
        odir  = 'tgnn_r'

    optimizer = AdaBound(model.parameters(), lr=1e-6, weight_decay=1e-8)
    # optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-8)
    criterion = torch.nn.L1Loss()

    for i in range(99):
        output_root = os.path.join(root_model, odir, '{:02d}'.format(i))
        if not os.path.isdir(output_root): 
            os.makedirs(output_root)
            break
    writer = SummaryWriter(output_root)

    for epoch in range(1, num_epochs+1):
        train_loss = tr.train(model, optimizer, train_data_loader, criterion)
        val_loss, val_rmse, _, _, _ = tr.test(model, val_data_loader, criterion)
        print('Epoch [{}/{}]\tTrain loss: {:.4f}\tVal loss: {:.4f} ({:.4f})'
                .format(epoch, num_epochs, train_loss, val_loss, val_rmse))

        writer.add_scalar('train/loss', train_loss, epoch)
        writer.add_scalar('train/MAE', train_loss, epoch)
        writer.add_scalar('valid/loss', val_loss, epoch)
        writer.add_scalar('valid/MAE', val_loss, epoch)

        if epoch%20 == 0:
            torch.save(model.state_dict(), os.path.join(output_root, 'model.{:05d}.pt'.format(epoch)))
        if epoch%10 == 0:
            _, _, idxs, targets, preds = tr.test(model, test_data_loader, criterion)
            numpy.savetxt(os.path.join(output_root, 'pred.{:05d}.txt'.format(epoch)), numpy.hstack([idxs, targets, preds]), delimiter=',')




            

100%|██████████| 10387/10387 [09:01<00:00, 19.18it/s]


Epoch [1/300]	Train loss: 1.8539	Val loss: 1.0065 (1.2785)
Epoch [2/300]	Train loss: 0.9246	Val loss: 0.8814 (1.1015)
Epoch [3/300]	Train loss: 0.8789	Val loss: 0.9009 (1.1710)
Epoch [4/300]	Train loss: 0.8613	Val loss: 0.8340 (1.0819)
Epoch [5/300]	Train loss: 0.8493	Val loss: 0.9106 (1.1205)
Epoch [6/300]	Train loss: 0.8431	Val loss: 0.8157 (1.0734)
Epoch [7/300]	Train loss: 0.8286	Val loss: 0.7969 (1.0803)
Epoch [8/300]	Train loss: 0.8071	Val loss: 0.7926 (1.0587)
Epoch [9/300]	Train loss: 0.7930	Val loss: 0.7890 (1.0841)
Epoch [10/300]	Train loss: 0.7928	Val loss: 0.7630 (1.0649)
Epoch [11/300]	Train loss: 0.7846	Val loss: 0.7678 (1.0503)
Epoch [12/300]	Train loss: 0.7770	Val loss: 0.7741 (1.0987)
Epoch [13/300]	Train loss: 0.7691	Val loss: 0.7413 (1.0706)
Epoch [14/300]	Train loss: 0.7788	Val loss: 0.7965 (1.0374)
Epoch [15/300]	Train loss: 0.7531	Val loss: 0.7800 (1.1523)
Epoch [16/300]	Train loss: 0.7480	Val loss: 0.7654 (1.1164)
Epoch [17/300]	Train loss: 0.7363	Val loss: 0.716